In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.shape

(50000, 2)

In [5]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [6]:
df.duplicated().sum()

418

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [9]:
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0}).infer_objects(copy=False)

/tmp/ipykernel_24/3452091296.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0}).infer_objects(copy=False)


In [10]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
df['sentiment'].value_counts()

sentiment
1    24884
0    24698
Name: count, dtype: int64

<!-- Text Cleaning
    Remove html tags
    remove special characters
    converting every thing to lower case 
    removing stop words
    stemming -->

In [12]:
# Text Cleaning
#     Remove html tags
#     remove special characters
#     converting every thing to lower case 
#     removing stop words
#     stemming

In [13]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def preprocess_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    text = ' '.join(filtered_words)
    
    # Stemming
    stemmer = PorterStemmer()
    stemmed_text = ' '.join([stemmer.stem(word) for word in text.split()])
    
    return stemmed_text



In [14]:
html_text = "<p>Hello <b>world</b>! This is <i>HTML</i> text.</p>"
cleaned_text = preprocess_text(html_text)
print(cleaned_text)

hello world html text


In [15]:
df['review'] = df['review'].apply(preprocess_text)

/tmp/ipykernel_24/2617301599.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [16]:
df.head()

,review,sentiment
0,one review mention watch 1 oz episod youll hoo...,1
1,wonder littl product film techniqu unassum old...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic there famili littl boy jake think there ...,0
4,petter mattei love time money visual stun film...,1


In [17]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
tfidf = TfidfVectorizer(max_features=2500)
# cv = CountVectorizer(max_features=3000)

In [18]:
# X = cv.fit_transform(df['review']).toarray()
X = tfidf.fit_transform(df['review']).toarray()

In [19]:
y = df.iloc[:,-1].values

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [21]:
X_train.shape

(39665, 2500)

In [22]:
X_test.shape

(9917, 2500)

In [23]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [24]:
GNB = GaussianNB()
MNB =  MultinomialNB()
BNB =  BernoulliNB()

In [25]:
GNB.fit(X_train, y_train)
MNB.fit(X_train, y_train)
BNB.fit(X_train, y_train)

BernoulliNB()

In [26]:
y_pred1 = GNB.predict(X_test)
y_pred2 = MNB.predict(X_test)
y_pred3 = BNB.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score, precision_score

In [28]:
print("GuassianNB", accuracy_score(y_test,y_pred1))
print("GuassianNB", precision_score(y_test,y_pred1))
print("MultinomialNB", accuracy_score(y_test,y_pred2))
print("MultinomialNB", precision_score(y_test,y_pred2))
print("BernoulliNB", accuracy_score(y_test,y_pred3))
print("BernoulliNB", precision_score(y_test,y_pred3))

GuassianNB 0.8011495411918927
GuassianNB 0.7939516129032258
MultinomialNB 0.8426943632146818
MultinomialNB 0.8282051282051283
BernoulliNB 0.8416859937481093
BernoulliNB 0.8307416267942583


In [29]:
# checking singlemessage
message ="I'm so disappointed with the quality of this product."



In [30]:
preprocessed_message = preprocess_text(message)
print(preprocessed_message)

im disappoint qualiti product


In [31]:
vector_input = tfidf.transform([preprocessed_message])


In [32]:
result = BNB.predict(vector_input)


In [33]:
if result == 1:
        print("Positive")
else:
    print('Negative')

Negative


In [34]:
# saving the model
import pickle
pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(MNB,open('model.pkl','wb'))
pickle.dump(preprocess_text, open('preprocess_text','wb'))